In [1]:
import os
import sys
import pickle
import psutil
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

# Define data path
DATA_PATH = "data"

In [2]:
pids = pickle.load(open(os.path.join(DATA_PATH,'pids.pkl'), 'rb'))
vids = pickle.load(open(os.path.join(DATA_PATH,'vids.pkl'), 'rb'))
targets = pickle.load(open(os.path.join(DATA_PATH,'targets.pkl'), 'rb'))
prob_targets = pickle.load(open(os.path.join(DATA_PATH,'prob_targets.pkl'), 'rb'))
prob_targets_allvisits = pickle.load(open(os.path.join(DATA_PATH,'prob_targets_allvisits.pkl'), 'rb'))
seqs = pickle.load(open(os.path.join(DATA_PATH,'seqs.pkl'), 'rb'))
diags = pickle.load(open(os.path.join(DATA_PATH,'diags.pkl'), 'rb'))
categories = pickle.load(open(os.path.join(DATA_PATH,'categories.pkl'), 'rb'))
sub_categories = pickle.load(open(os.path.join(DATA_PATH,'subcategories.pkl'), 'rb'))
codes = pickle.load(open(os.path.join(DATA_PATH,'icd9.pkl'), 'rb'))
assert len(pids) == len(vids) == len(targets) == len(seqs)

In [4]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    
    def __init__(self, seqs, targets):
        
        """
        TODO: Store `seqs`. to `self.x` and `hfs` to `self.y`.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        Do NOT permute the data.
        """
        self.x = seqs
        self.y = targets
    
    def __len__(self):
        
        """
        TODO: Return the number of samples (i.e. patients).
        """
        
        return(len(self.x))
    
    def __getitem__(self, index):
        
        """
        TODO: Generates one sample of data.
        
        Note that you DO NOT need to covert them to tensor as we will do this later.
        """
        return (self.x[index], self.y[index])

In [5]:
dataset = CustomDataset(seqs, prob_targets_allvisits)

In [6]:
def collate_fn(data):
    """
    Arguments:
        data: a list of samples fetched from `CustomDataset`
        
    Outputs:
        x: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.long
        masks: a tensor of shape (# patiens, max # visits, max # diagnosis codes) of type torch.bool
        rev_x: same as x but in reversed time. This will be used in our RNN model for masking 
        rev_masks: same as mask but in reversed time. This will be used in our RNN model for masking
        y: a tensor of shape (# patiens) of type torch.float
        
    Note that you can obtains the list of diagnosis codes and the list of hf labels
        using: `sequences, labels = zip(*data)`
    """
    sequences, targets = zip(*data)

#     y = torch.tensor(targets, dtype=torch.float)
    #import pdb; pdb.set_trace()
    num_patients = len(sequences)
    num_visits = [len(patient) for patient in sequences]
    num_codes = [len(visit) for patient in sequences for visit in patient]
    num_categories = len(targets[0][0])

    max_num_visits = max(num_visits)
    max_num_codes = max(num_codes)
    
    sum_visits = sum(num_visits)
    
    x = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.int)
    y = torch.zeros((sum_visits - num_patients, num_categories), dtype=torch.float32)
    x_masks = torch.zeros((sum_visits - num_patients, max_num_codes), dtype=torch.bool)

#     for i_patient, patient in enumerate(sequences):   
#         for j_visit, visit in enumerate(patient):
#             """
#             TODO: update `x`, `rev_x`, `masks`, and `rev_masks`
#             """ 
#             x[i_patient, j_visit] = torch.Tensor(visit)
#             #x_masks[i_patient, j_visit] = torch.Tensor(np.ones(num_codes, dtype=int))
#             x_masks[i_patient, j_visit] = 1
#     import pdb; pdb.set_trace()
    n = 0
    for i,patient in enumerate(sequences):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            for k,code in enumerate(visit):
                x[n,k] = code
                x_masks[n,k] = 1
            n+=1
    n = 0
    for i,patient in enumerate(targets):
        for j,visit in enumerate(patient):
            if j == len(patient) - 1:
                break
            y[n] = torch.tensor(patient[j+1])
            n += 1
    
    
    return x, x_masks, y

In [7]:
train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)
val_split = int(len(dataset)*0.10)

In [8]:
from torch.utils.data.dataset import random_split

train_split = int(len(dataset)*0.75)
test_split = int(len(dataset)*0.15)

lengths = [train_split, test_split, len(dataset) - (train_split + test_split)]
train_dataset, test_dataset, val_dataset = random_split(dataset, lengths)

print("Length of train dataset:", len(train_dataset))
print("Length of test dataset:", len(test_dataset))
print("Length of val dataset:", len(val_dataset))

Length of train dataset: 6561
Length of test dataset: 1312
Length of val dataset: 875


In [9]:
from torch.utils.data import DataLoader

def load_data(train_dataset, test_dataset, val_dataset, collate_fn):
    
    '''
    
    Arguments:
        train dataset: train dataset of type `CustomDataset`
        val dataset: validation dataset of type `CustomDataset`
        collate_fn: collate function
        
    Outputs:
        train_loader, val_loader: train and validation dataloaders
    
    Note that you need to pass the collate function to the data loader `collate_fn()`.
    '''
    
    batch_size = 100
    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size,
                                               collate_fn=collate_fn,
                                               shuffle=False)
    test_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           collate_fn=collate_fn,
                                           shuffle=False)
    val_loader = torch.utils.data.DataLoader(val_dataset,
                                             batch_size=batch_size,
                                             collate_fn=collate_fn,
                                             shuffle=False)
    
    return train_loader, test_loader, val_loader


train_loader, test_loader, val_loader = load_data(train_dataset, test_dataset, val_dataset, collate_fn)

In [10]:
def sum_embeddings_with_mask(x, masks):
    """
    Mask select the embeddings for true visits (not padding visits) and then sum the embeddings for each visit up.

    Arguments:
        x: the embeddings of diagnosis sequence of shape (batch_size, # visits, # diagnosis codes, embedding_dim)
        masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

    Outputs:
        sum_embeddings: the sum of embeddings of shape (batch_size, # visits, embedding_dim)
    """
    
    x = x * masks.unsqueeze(-1)
    x = torch.sum(x, dim = -2)
    return x

In [11]:
def indices_to_multihot(indices, masks, dim):
    #import pdb; pdb.set_trace()
    #indices = indices[masks.any(dim=1)]
    multihot = torch.zeros((indices.shape[0], dim), dtype=torch.int)
    for idx, row in enumerate(indices):
        y_idx = row[masks[idx]].unique()
        multihot[idx] = F.one_hot(y_idx.to(torch.int64), multihot.shape[1]).sum(0)
    return multihot

In [39]:
class BaselineMLP(nn.Module):
    
    """
    TODO: implement the naive RNN model above.
    """
    
    def __init__(self, num_codes, num_categories):
        super().__init__()
        """
        Arguments:
            num_codes: total number of diagnosis codes
        """
#         self.padding_idx = 0
        self.embedding = nn.Embedding(num_codes, embedding_dim=128, padding_idx=0)
        self.fc = nn.Linear(128, num_categories)
        self.softmax = nn.Softmax(dim=-1)
        
    
    def forward(self, x, masks):
        """
        Arguments:
            x: the diagnosis sequence of shape (batch_size, # visits, # diagnosis codes)
            masks: the padding masks of shape (batch_size, # visits, # diagnosis codes)

        Outputs:
            probs: probabilities of shape (batch_size)
        """
#         import pdb; pdb.set_trace()
#         num_codes = self.embedding.weight.shape[0]
#         x = indices_to_multihot(x, masks, num_codes)
#         x[~masks] = self.padding_idx
        x[masks] += 1
        x = self.embedding(x)
        x = x.sum(dim=1)
        #x = sum_embeddings_with_mask(x, masks)
        logits = self.fc(x)
#         logits = logits.mean(dim=1)
        probs = self.softmax(logits)
        return logits
    

# load the model here
baseline_mlp = BaselineMLP(num_codes = len(codes), num_categories=len(sub_categories))
baseline_mlp

BaselineMLP(
  (embedding): Embedding(4903, 128, padding_idx=0)
  (fc): Linear(in_features=128, out_features=184, bias=True)
  (softmax): Softmax(dim=-1)
)

In [40]:
criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(baseline_mlp.parameters(), lr=0.001)
optimizer = torch.optim.Adadelta(baseline_mlp.parameters(), weight_decay=0.001)

In [59]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score


def eval_model(model, test_loader, k=15, n=-1):
    
    """
    Arguments:
        model: the RNN model
        val_loader: validation dataloader
        
    Outputs:
        precision: overall precision score
        recall: overall recall score
        f1: overall f1 score
        roc_auc: overall roc_auc score
        
    """
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    all_precision = []
    all_accuracy = []
    
    model.eval()
    with torch.no_grad():
        for x, masks, y in test_loader:
#             import pdb; pdb.set_trace()
            n_eval = y.shape[0] - 1 if n == -1 else n
            y_hat = model(x, masks)
            y_hat = F.softmax(y_hat, dim=-1)
#             num_labels = y_hat.shape[1]
#             num_categories = torch.count_nonzero(y, dim=1)
            nz_rows, nz_cols = torch.nonzero(y, as_tuple=True)
            k_correct = 0
#             predictions = 0
            total_precision = 0
            total_accuracy = 0
            for i in range(n_eval):
                visit_correct = 0
                y_true = nz_cols[nz_rows == i]
                _, y_pred = torch.topk(y_hat[i], k)
#                 for v in y_pred:
#                     if v in y_true:
#                         visit_correct += 1
                for v in y_true:
                    if v in y_pred:
                        visit_correct += 1
#                 predictions += len(y_true)
                visit_precision = visit_correct / min(k, len(y_true))
                visit_accuracy = visit_correct / len(y_true)
                #print(f'visit {i}: precision: {visit_precision:0.2f} accuracy: {visit_accuracy:0.2f}')
                k_correct += visit_correct
                total_precision += visit_precision
                total_accuracy += visit_accuracy
            #import pdb; pdb.set_trace()
#             precision_k = precision / k
#             accuracy_k = k_correct / predictions
            precision_k = total_precision / n_eval
            accuracy_k = total_accuracy / n_eval
            all_precision.append(precision_k)
            all_accuracy.append(accuracy_k)
            
#             y_score = torch.cat((y_score,  y_hat.detach().to('cpu')), dim=0)
#             y_hat = (y_hat > 0.5).int()
#             y_pred = torch.cat((y_pred,  y_hat.detach().to('cpu')), dim=0)
#             y_true = torch.cat((y_true, y.detach().to('cpu')), dim=0)
#     import pdb; pdb.set_trace()
    total_precision_k = np.mean(all_precision)
    total_accuracy_k = np.mean(all_accuracy)
    return total_precision_k, total_accuracy_k

In [53]:
def train(model, train_loader, test_loader, n_epochs):
    """

    """
    base_cpu, base_ram = print_cpu_usage()
    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
#         import pdb; pdb.set_trace()
        for x, masks, y in train_loader:

            y_hat = model(x, masks)
#             mask_idxs = masks.sum(dim=1) - 1
#             y_hat = y_hat[range(len(masks)), mask_idxs]
            loss = criterion(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss = train_loss / len(train_loader)
        print_cpu_usage()
        print(f'Epoch: {epoch+1} \t Training Loss: {train_loss:.6f}')
        for k in range(5, 31, 5):
            precision_k, accuracy_k = eval_model(model, val_loader, k=k)
            print(f'Epoch: {epoch+1} \t Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')
    final_cpu, final_ram = print_cpu_usage()

In [43]:
def print_cpu_usage():
    load = psutil.getloadavg()[2]
    cpu_usage = (load/os.cpu_count()) * 100
    ram = psutil.virtual_memory()[2]
    print(f"CPU: {cpu_usage:0.2f}")
    print(f"RAM %: {ram}")
    return cpu_usage, ram

In [44]:
n_epochs = 100
%time train(baseline_mlp, train_loader, test_loader, n_epochs)

CPU: 12.32
RAM %: 63.1
CPU: 12.32
RAM %: 63.1
Epoch: 1 	 Training Loss: 4.771595
Epoch: 1 	 Validation precision@k5: 0.5344, accuracy@k5: 0.2719
Epoch: 1 	 Validation precision@k10: 0.5039, accuracy@k10: 0.4238
Epoch: 1 	 Validation precision@k15: 0.5470, accuracy@k15: 0.5286
Epoch: 1 	 Validation precision@k20: 0.6132, accuracy@k20: 0.6103
Epoch: 1 	 Validation precision@k25: 0.6761, accuracy@k25: 0.6758
Epoch: 1 	 Validation precision@k30: 0.7260, accuracy@k30: 0.7260
CPU: 12.35
RAM %: 63.1
Epoch: 2 	 Training Loss: 3.892900
Epoch: 2 	 Validation precision@k5: 0.5961, accuracy@k5: 0.3053
Epoch: 2 	 Validation precision@k10: 0.5551, accuracy@k10: 0.4689
Epoch: 2 	 Validation precision@k15: 0.5969, accuracy@k15: 0.5774
Epoch: 2 	 Validation precision@k20: 0.6643, accuracy@k20: 0.6611
Epoch: 2 	 Validation precision@k25: 0.7235, accuracy@k25: 0.7232
Epoch: 2 	 Validation precision@k30: 0.7728, accuracy@k30: 0.7728
CPU: 12.35
RAM %: 62.6
Epoch: 3 	 Training Loss: 3.782546
Epoch: 3 	 Vali

CPU: 12.67
RAM %: 62.7
Epoch: 19 	 Training Loss: 3.582549
Epoch: 19 	 Validation precision@k5: 0.6535, accuracy@k5: 0.3357
Epoch: 19 	 Validation precision@k10: 0.6068, accuracy@k10: 0.5129
Epoch: 19 	 Validation precision@k15: 0.6510, accuracy@k15: 0.6299
Epoch: 19 	 Validation precision@k20: 0.7123, accuracy@k20: 0.7091
Epoch: 19 	 Validation precision@k25: 0.7717, accuracy@k25: 0.7714
Epoch: 19 	 Validation precision@k30: 0.8166, accuracy@k30: 0.8166
CPU: 12.63
RAM %: 62.5
Epoch: 20 	 Training Loss: 3.576729
Epoch: 20 	 Validation precision@k5: 0.6557, accuracy@k5: 0.3367
Epoch: 20 	 Validation precision@k10: 0.6093, accuracy@k10: 0.5148
Epoch: 20 	 Validation precision@k15: 0.6532, accuracy@k15: 0.6320
Epoch: 20 	 Validation precision@k20: 0.7136, accuracy@k20: 0.7104
Epoch: 20 	 Validation precision@k25: 0.7724, accuracy@k25: 0.7721
Epoch: 20 	 Validation precision@k30: 0.8174, accuracy@k30: 0.8174
CPU: 12.63
RAM %: 62.4
Epoch: 21 	 Training Loss: 3.571130
Epoch: 21 	 Validation 

Epoch: 36 	 Validation precision@k30: 0.8309, accuracy@k30: 0.8309
CPU: 13.10
RAM %: 62.9
Epoch: 37 	 Training Loss: 3.508199
Epoch: 37 	 Validation precision@k5: 0.6816, accuracy@k5: 0.3503
Epoch: 37 	 Validation precision@k10: 0.6336, accuracy@k10: 0.5351
Epoch: 37 	 Validation precision@k15: 0.6727, accuracy@k15: 0.6505
Epoch: 37 	 Validation precision@k20: 0.7301, accuracy@k20: 0.7267
Epoch: 37 	 Validation precision@k25: 0.7876, accuracy@k25: 0.7873
Epoch: 37 	 Validation precision@k30: 0.8317, accuracy@k30: 0.8317
CPU: 13.35
RAM %: 62.6
Epoch: 38 	 Training Loss: 3.505800
Epoch: 38 	 Validation precision@k5: 0.6832, accuracy@k5: 0.3515
Epoch: 38 	 Validation precision@k10: 0.6338, accuracy@k10: 0.5352
Epoch: 38 	 Validation precision@k15: 0.6740, accuracy@k15: 0.6518
Epoch: 38 	 Validation precision@k20: 0.7304, accuracy@k20: 0.7271
Epoch: 38 	 Validation precision@k25: 0.7890, accuracy@k25: 0.7887
Epoch: 38 	 Validation precision@k30: 0.8326, accuracy@k30: 0.8326
CPU: 13.31
RAM 

Epoch: 54 	 Validation precision@k25: 0.7924, accuracy@k25: 0.7920
Epoch: 54 	 Validation precision@k30: 0.8361, accuracy@k30: 0.8361
CPU: 13.75
RAM %: 62.8
Epoch: 55 	 Training Loss: 3.483120
Epoch: 55 	 Validation precision@k5: 0.6923, accuracy@k5: 0.3568
Epoch: 55 	 Validation precision@k10: 0.6422, accuracy@k10: 0.5423
Epoch: 55 	 Validation precision@k15: 0.6786, accuracy@k15: 0.6561
Epoch: 55 	 Validation precision@k20: 0.7401, accuracy@k20: 0.7367
Epoch: 55 	 Validation precision@k25: 0.7924, accuracy@k25: 0.7920
Epoch: 55 	 Validation precision@k30: 0.8360, accuracy@k30: 0.8360
CPU: 13.75
RAM %: 62.5
Epoch: 56 	 Training Loss: 3.482477
Epoch: 56 	 Validation precision@k5: 0.6924, accuracy@k5: 0.3568
Epoch: 56 	 Validation precision@k10: 0.6425, accuracy@k10: 0.5426
Epoch: 56 	 Validation precision@k15: 0.6784, accuracy@k15: 0.6560
Epoch: 56 	 Validation precision@k20: 0.7402, accuracy@k20: 0.7368
Epoch: 56 	 Validation precision@k25: 0.7928, accuracy@k25: 0.7925
Epoch: 56 	 Val

Epoch: 72 	 Validation precision@k20: 0.7396, accuracy@k20: 0.7363
Epoch: 72 	 Validation precision@k25: 0.7944, accuracy@k25: 0.7940
Epoch: 72 	 Validation precision@k30: 0.8357, accuracy@k30: 0.8357
CPU: 14.83
RAM %: 63.2
Epoch: 73 	 Training Loss: 3.476237
Epoch: 73 	 Validation precision@k5: 0.6937, accuracy@k5: 0.3582
Epoch: 73 	 Validation precision@k10: 0.6434, accuracy@k10: 0.5436
Epoch: 73 	 Validation precision@k15: 0.6788, accuracy@k15: 0.6564
Epoch: 73 	 Validation precision@k20: 0.7397, accuracy@k20: 0.7363
Epoch: 73 	 Validation precision@k25: 0.7941, accuracy@k25: 0.7937
Epoch: 73 	 Validation precision@k30: 0.8359, accuracy@k30: 0.8359
CPU: 14.81
RAM %: 62.7
Epoch: 74 	 Training Loss: 3.476045
Epoch: 74 	 Validation precision@k5: 0.6937, accuracy@k5: 0.3583
Epoch: 74 	 Validation precision@k10: 0.6435, accuracy@k10: 0.5437
Epoch: 74 	 Validation precision@k15: 0.6789, accuracy@k15: 0.6565
Epoch: 74 	 Validation precision@k20: 0.7396, accuracy@k20: 0.7362
Epoch: 74 	 Val

Epoch: 90 	 Validation precision@k15: 0.6792, accuracy@k15: 0.6568
Epoch: 90 	 Validation precision@k20: 0.7401, accuracy@k20: 0.7368
Epoch: 90 	 Validation precision@k25: 0.7928, accuracy@k25: 0.7925
Epoch: 90 	 Validation precision@k30: 0.8361, accuracy@k30: 0.8361
CPU: 15.29
RAM %: 62.9
Epoch: 91 	 Training Loss: 3.474132
Epoch: 91 	 Validation precision@k5: 0.6939, accuracy@k5: 0.3582
Epoch: 91 	 Validation precision@k10: 0.6430, accuracy@k10: 0.5431
Epoch: 91 	 Validation precision@k15: 0.6792, accuracy@k15: 0.6568
Epoch: 91 	 Validation precision@k20: 0.7401, accuracy@k20: 0.7368
Epoch: 91 	 Validation precision@k25: 0.7928, accuracy@k25: 0.7925
Epoch: 91 	 Validation precision@k30: 0.8361, accuracy@k30: 0.8361
CPU: 15.23
RAM %: 62.8
Epoch: 92 	 Training Loss: 3.474073
Epoch: 92 	 Validation precision@k5: 0.6939, accuracy@k5: 0.3582
Epoch: 92 	 Validation precision@k10: 0.6428, accuracy@k10: 0.5430
Epoch: 92 	 Validation precision@k15: 0.6793, accuracy@k15: 0.6569
Epoch: 92 	 Val

In [60]:
for k in range(5, 31, 5):
    precision_k, accuracy_k = eval_model(baseline_mlp, test_loader, k=k)
    print(f'Validation precision@k{k}: {precision_k:.4f}, accuracy@k{k}: {accuracy_k:.4f}')

Validation precision@k5: 0.7440, accuracy@k5: 0.3867
Validation precision@k10: 0.6949, accuracy@k10: 0.5838
Validation precision@k15: 0.7231, accuracy@k15: 0.6964
Validation precision@k20: 0.7735, accuracy@k20: 0.7691
Validation precision@k25: 0.8214, accuracy@k25: 0.8211
Validation precision@k30: 0.8596, accuracy@k30: 0.8596
